# Final

## Versión Gratis

In [1]:
import chromadb
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader


embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis
persistent_client = chromadb.PersistentClient(path='./vectordb_gratis')

/tmp/ipykernel_969740/2869466478.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis


In [3]:
def add_files_to_vectordb(filepath):
    loader = PyPDFLoader(filepath)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb_gratis')

In [22]:
add_files_to_vectordb('./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf')

In [5]:
add_files_to_vectordb('./data/los7HabitosGenteAltamenteEfectiva.pdf')

## Ver si se agregaron

In [6]:
def get_unique_sources_list(chroma_settings):
    # Obtén los datos de la colección
    collection_data = chroma_settings.get_collection('langchain').get(include=['embeddings', 'documents', 'metadatas'])
    
    # Extrae los metadatos
    metadatas = collection_data['metadatas']
    
    # Obtén los valores únicos de 'source'
    sources = set()
    for metadata in metadatas:
        source = metadata.get('source', None)
        if source:
            sources.add(source)
    
    # Obtener solo el nombre de archivo de cada ruta
    file_names = list(set(source.split('/')[-1] for source in sources))
    
    return file_names

In [7]:
get_unique_sources_list(persistent_client)

['los7HabitosGenteAltamenteEfectiva.pdf',
 'CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf']

## Version paga

In [ ]:
from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [47]:
import os
import re
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


persistent_client = chromadb.PersistentClient(path='./vectordb')

In [48]:
# ==========================
# 3. Función de limpieza Adicionada VM
# ==========================
def clean_text(text: str) -> str:
    patrones = [
        r"FORMATO REQUISICIÓN.*",
        r"ECOPETROL DESARROLLO DE PROYECTO.*",
        r"Todos los derechos reservados.*",
        r"___________________________________________________________________",
        r"EDP-F-046.*Versión.*",
        r"MR.*#:.*CAS.*",
    ]
    for p in patrones:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"\s{2,}", " ", text)  # normalizar espacios
    return text.strip()

In [49]:
# ==========================
# 4. Función principal Modificada PUEBA!!!!!!!!!!!!! VM
# ==========================
def add_files_to_vectordb(filepath):
    # Cargar PDF
    loader = PyPDFLoader(filepath)
    docs = loader.load()

    # Limpiar cada página
    for d in docs:
        d.page_content = clean_text(d.page_content)

    # Dividir en chunks optimizados
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    splits = text_splitter.split_documents(docs)

    # Crear/actualizar vectorstore
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory="./vectordb"
    )

    # ⚠️ Ya no se usa .persist() en la nueva API
    print(f"✅ {len(splits)} chunks limpios guardados en ./vectordb")
    return vectorstore


In [ ]:
# VERSION OROGINAL

def add_files_to_vectordb(filepath):
    loader = PyPDFLoader(filepath)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory='./vectordb')

In [50]:
add_files_to_vectordb('./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf')

✅ 359 chunks limpios guardados en ./vectordb


In [12]:
add_files_to_vectordb('./data/los7HabitosGenteAltamenteEfectiva.pdf')

## Ver si se agregaron

In [51]:
def get_unique_sources_list(chroma_settings):
    # Obtén los datos de la colección
    collection_data = chroma_settings.get_collection('langchain').get(include=['embeddings', 'documents', 'metadatas'])
    
    # Extrae los metadatos
    metadatas = collection_data['metadatas']
    
    # Obtén los valores únicos de 'source'
    sources = set()
    for metadata in metadatas:
        source = metadata.get('source', None)
        if source:
            sources.add(source)
    
    # Obtener solo el nombre de archivo de cada ruta
    file_names = list(set(source.split('/')[-1] for source in sources))
    
    return file_names

In [52]:
get_unique_sources_list(persistent_client)

['CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf']

In [55]:
#PRUEBAS ADICIONALES VM

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

def load_vectordb(persist_dir="./vectordb"):
    return Chroma(
        persist_directory=persist_dir,
        embedding_function=OpenAIEmbeddings()
    )

def create_rag(vectorstore):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un asistente que responde preguntas con base en el siguiente contexto:
        {context} 
        Si no sabes la respuesta, di que no lo sabes.
        Responde en máximo tres oraciones, de forma concisa."""),
        ("human", "{input}")
    ])

    chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(vectorstore.as_retriever(), chain)

# ========================
# Ejemplo de consulta
# ========================
vectorstore = load_vectordb("./vectordb")
rag = create_rag(vectorstore)

pregunta = "¿Cuál es el alcance del suministro?"
respuesta = rag.invoke({"input": pregunta})
print("🔎 Pregunta:", pregunta)
print("💡 Respuesta:", respuesta["answer"])


🔎 Pregunta: ¿Cuál es el alcance del suministro?
💡 Respuesta: El alcance del suministro incluye una unidad tipo paquete de inyección de químicos y quills de inyección roscados, cumpliendo con todos los requisitos establecidos en la requisición y sus anexos. Además, el proveedor debe garantizar asistencia técnica en Colombia y disponibilidad de partes de reemplazo. El suministro debe entregarse en correcto funcionamiento y a satisfacción de ECOPETROL S.A.
